In [ ]:
import pandas as pd
import requests as r
import json


In [6]:
#setting konfigurasi
with open("../env.json", "r", encoding="utf-8") as file:
    config = json.load(file)  # Membaca dan parsing JSON


In [17]:
#koneksi database
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
engine = create_engine((f"mysql+pymysql://{config['user']}:%s@{config['ip']}:3306/{config['database']}")% quote_plus(config['pass'])) #server

In [9]:
#Get Daftar Data Dinamis BPS Bali
def getVarByDomain(domain='5100'):
    global config
    var_pages=[]
    var = r.get(f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/key/{config['token']}/')
    pages=var.json()['data'][0]['pages']
    current_page = 1
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/page/{current_page}/key/{config['token']}/'
        var_page=r.get(url)
        var_pages.append(pd.json_normalize(var_page.json()['data'][1]))
        current_page=current_page+1
    var_pages_df = pd.concat(var_pages)
    return var_pages_df

In [10]:
#Get Daftar Periode
def getPriodeByVar(domain='5100',varId='106'):
    global config
    current_page = 1
    url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
    dt_json = r.get(url)
    pages = dt_json.json()['data'][0]['pages']
    dt_all = []
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
        dt = pd.json_normalize(dt_json.json()['data'][1])
        dt_all.append(dt)
        current_page = current_page+1
    dt_all = pd.concat(dt_all)
    dt_all['varId']=varId
    return dt_all

In [12]:
def crossJoin(left,right):
    if left.empty:
        left = right
    else:
        left = pd.merge(left,right,how='cross')
    return left
def getDataByVarId(varId='106',domain='5100'):
    global config
    #Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk
    dt_th = getPriodeByVar(domain=domain,varId=varId)
    dt_all = []
    for i in dt_th.index:
        try:
            th_id = dt_th.iloc[i]['th_id']
            url = f'https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain}/var/{varId}/key/{config['token']}/th/{th_id}/'
            dt_json = r.get(url)
            attr = pd.DataFrame()
            if 'vervar' in dt_json.json().keys():
                vervar=pd.json_normalize(dt_json.json()['vervar'])
                vervar=vervar.rename(columns={'val': 'verval_val','label': 'vervar_label'})
                attr = crossJoin(attr,vervar)
            if 'var' in dt_json.json().keys():
                var=pd.json_normalize(dt_json.json()['var'])
                var=var.rename(columns={'val': 'var_val','label': 'var_label'})
                var=var[['var_val']]
                attr = crossJoin(attr,var)
            if 'turvar' in dt_json.json().keys():
                turvar=pd.json_normalize(dt_json.json()['turvar'])
                turvar=turvar.rename(columns={'val': 'turvar_val','label': 'turvar_label'})
                attr = crossJoin(attr,turvar)
            if 'tahun' in dt_json.json().keys():
                tahun=pd.json_normalize(dt_json.json()['tahun'])
                tahun=tahun.rename(columns={'val': 'tahun_val','label': 'tahun_label'})
                attr = crossJoin(attr,tahun)
            if 'turtahun' in dt_json.json().keys():
                turtahun=pd.json_normalize(dt_json.json()['turtahun'])
                turtahun=turtahun.rename(columns={'val': 'turtahun_val','label': 'turtahun_label'})
                attr = crossJoin(attr,turtahun)
            val_columns = [col for col in attr.columns if col.endswith('val')]
            attr['id']=attr[val_columns].astype('str').agg(''.join, axis=1)
            dt_content=pd.json_normalize(dt_json.json()['datacontent']).transpose().reset_index().rename(columns={'index':'content_val',0:'content_label'})
            attr = pd.merge(attr,dt_content,left_on='id',right_on='content_val',how='left')
            attr = attr[attr.content_label.notna()]
            dt_all.append(attr)
        except Exception as e:
            print(e)
    dt_all=pd.concat(dt_all)
    return dt_all

In [18]:
daftar_data=getVarByDomain(domain='5100')
#daftar_data.to_excel('daftar_data_web_api_dinamis.xlsx')
#Contoh akan mengambil data by id dari daftar data
data=getDataByVarId(domain='5100',varId='47')

In [ ]:
#proses write db
nama_tabel = "inflasi_singaraja"
data.to_sql(nama_tabel,con=engine,if_exists='replace',index=False) # lihat panduan di pandas to_sql

48

In [20]:
#proses read data
dt = pd.read_sql(nama_tabel,con=engine,dtype=str) # lihat panduan di pandas to_sql

In [21]:
dt

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Bahan Makanan,47,0,Tidak ada,119,2019,0,Tahun,14701190,14701190,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",47,0,Tidak ada,119,2019,0,Tahun,24701190,24701190,3.82
2,3,Perumahan,47,0,Tidak ada,119,2019,0,Tahun,34701190,34701190,0.60
3,4,Sandang,47,0,Tidak ada,119,2019,0,Tahun,44701190,44701190,4.02
4,5,Kesehatan,47,0,Tidak ada,119,2019,0,Tahun,54701190,54701190,5.67
5,6,"Pendidikan, Rekreasi, dan Olahraga",47,0,Tidak ada,119,2019,0,Tahun,64701190,64701190,6.28
6,7,Transpor dan Komunikasi,47,0,Tidak ada,119,2019,0,Tahun,74701190,74701190,1.17
7,8,<b> Umum </b>,47,0,Tidak ada,119,2019,0,Tahun,84701190,84701190,2.42
8,1,Bahan Makanan,47,0,Tidak ada,118,2018,0,Tahun,14701180,14701180,4.33
9,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",47,0,Tidak ada,118,2018,0,Tahun,24701180,24701180,0.77


In [ ]:
### id 47 Inflasi Tahunan Kota Singaraja Menurut 7 Kelompok Pengeluaran
### korelasi dengan web BPS
### modifikasi ke bentuk seperti ini 
# https://bali.bps.go.id/id/statistics-table/2/NDcjMg==/inflasi-tahunan-kota-singaraja-menurut-7-kelompok-pengeluaran.html

In [ ]:
#proses cleaning
data=data[['verval_val','vervar_label','tahun_label','content_label']]
data["vervar_label"] = data["vervar_label"].str.replace(r"<.*?>", "", regex=True)
#proses rename colomn
data_tb=data.rename(columns={'vervar_label' : 'kelompok_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data_tb=data_tb.pivot_table(index=['verval_val','kelompok_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data_tb.columns.values]
data_tb=data_tb.reset_index()

In [ ]:
#contoh penggunaan 

In [52]:
data_tb=data_tb[[ 'kelompok_inflasi','inflasi_2014','inflasi_2015','inflasi_2016','inflasi_2017','inflasi_2018','inflasi_2019']]

In [53]:
data_tb

,kelompok_inflasi,inflasi_2014,inflasi_2015,inflasi_2016,inflasi_2017,inflasi_2018,inflasi_2019
0,Bahan Makanan,8.09,4.74,9.98,-3.39,4.33,1.97
1,"Makanan Jadi, Minuman, Rokok, dan Tembakau",15.00,4.22,4.08,3.15,0.77,3.82
2,Perumahan,9.97,3.33,1.83,8.44,-3.07,0.60
3,Sandang,6.25,7.73,8.47,6.53,4.42,4.02
4,Kesehatan,1.45,2.61,4.65,5.02,2.67,5.67
5,"Pendidikan, Rekreasi, dan Olahraga",9.37,3.29,6.24,7.48,10.51,6.28
6,Transpor dan Komunikasi,13.36,-5.23,-3.48,5.85,4.16,1.17
7,Umum,10.32,2.97,4.57,3.38,1.88,2.42


In [63]:
data.to_excel('data_47.xlsx')